In [32]:
import preprocess.prepare as prepare
import preprocess.word_bag as word_bag
import model.naive_bayes
import model.nn
import model.trivial
import numpy as np
from utils.cache import cached
from utils.data import train_dev_split
from metrics.hamming import hamming_distance
from model.trivial import AlwaysFalseModel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = cached(lambda:prepare.CratesData(), "crates_data.pkl")

Loading cached object from .cs229_cache\crates_data.pkl


In [3]:
crates = [crate for crate in data.id2crates.values() if crate.category_indices]
len(crates)

32225

In [4]:
len(crates)

32225

In [5]:
dic = cached(lambda:word_bag.create_dictionary(crates), "dictionary.pkl")

Loading cached object from .cs229_cache\dictionary.pkl


In [11]:
X, y = cached(lambda: word_bag.transform_crate(crates[:10000], dic, len(data.categories)), "wb10000.pkl")
X = X.astype(np.float32)
y = y.astype(np.float32)
X_train, y_train, X_dev, y_dev = train_dev_split(X, y, train_ratio=0.8, seed=0)

Loading cached object from .cs229_cache\wb10000.pkl


In [34]:
model = model.nn.DNNModel(num_categories=len(data.categories), learning_rate=0.0001, reg=0.000)

In [35]:
model.fit(X_train, y_train, X_dev, y_dev, epochs=20)

Epoch: 0, Loss: 0.0070, val_loss: 0.0069, Hamming Distance: 0.0815, val_hamming_dist: 0.0804
Epoch: 1, Loss: 0.0047, val_loss: 0.0034, Hamming Distance: 0.0366, val_hamming_dist: 0.0366
Epoch: 2, Loss: 0.0023, val_loss: 0.0021, Hamming Distance: 0.0221, val_hamming_dist: 0.0219
Epoch: 3, Loss: 0.0018, val_loss: 0.0021, Hamming Distance: 0.0215, val_hamming_dist: 0.0216
Epoch: 4, Loss: 0.0016, val_loss: 0.0019, Hamming Distance: 0.0206, val_hamming_dist: 0.0207
Epoch: 5, Loss: 0.0016, val_loss: 0.0030, Hamming Distance: 0.0216, val_hamming_dist: 0.0216
Epoch: 6, Loss: 0.0022, val_loss: 0.0022, Hamming Distance: 0.0290, val_hamming_dist: 0.0299
Epoch: 7, Loss: 0.0015, val_loss: 0.0020, Hamming Distance: 0.0208, val_hamming_dist: 0.0211
Epoch: 8, Loss: 0.0014, val_loss: 0.0021, Hamming Distance: 0.0197, val_hamming_dist: 0.0203
Epoch: 9, Loss: 0.0014, val_loss: 0.0018, Hamming Distance: 0.0190, val_hamming_dist: 0.0202
Epoch: 10, Loss: 0.0013, val_loss: 0.0017, Hamming Distance: 0.0179, v

In [36]:
nn_pred_train = model.predict(X_train)
nn_pred_dev = model.predict(X_dev)
nn_train_hamming = hamming_distance(y_train, nn_pred_train)
nn_dev_hamming = hamming_distance(y_dev, nn_pred_dev)
print("NN Train Hamming: ", nn_train_hamming)
print("NN Dev Hamming: ", nn_dev_hamming)

NN Train Hamming:  0.008004518072289158
NN Dev Hamming:  0.017981927710843375


In [37]:
a0_pred_train = AlwaysFalseModel(len(data.categories)).predict(X_train)
a0_pred_dev = AlwaysFalseModel(len(data.categories)).predict(X_dev)
a0_train_hamming = hamming_distance(y_train, a0_pred_train)
a0_dev_hamming = hamming_distance(y_dev, a0_pred_dev)
print("Always False Train Hamming: ", a0_train_hamming)
print("Always False Dev Hamming: ", a0_dev_hamming)

Always False Train Hamming:  0.02208132530120482
Always False Dev Hamming:  0.021921686746987953
